<a href="https://colab.research.google.com/github/ALHellen/Tech-Challenge/blob/main/TechChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets accelerate peft trl bitsandbytes

import json
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig


In [ ]:
def valor_vazio(valor):
    return valor in ({}, [], None, "")

In [ ]:
def remove_se_vazio(produto):
    if isinstance(produto, dict):
        limpo = {}
        for chave, valor in produto.items():
            val = remove_se_vazio(valor)
            if not valor_vazio(val):  # mantém apenas valores não vazios
                limpo[chave] = val
        return limpo

    elif isinstance(produto, list):
        nova_lista = [remove_se_vazio(v) for v in produto if not valor_vazio(v)]
        return nova_lista

    else:
        return produto

In [ ]:
def limpar_json(input_file, output_file):
    objeto_limpo = []

    with open(input_file, "r", encoding="utf-8") as f:
        for linha in f:
            linha = linha.strip()
            if not linha:
                continue
            try:
                obj = json.loads(linha)
                limpo = remove_se_vazio(obj)
                if limpo:  # mantém se ainda sobrou algo
                    objeto_limpo.append(limpo)
            except json.JSONDecodeError:
                print(f"⚠️ Linha inválida ignorada: {linha[:80]}...")

    # mantém só colunas desejadas
    colunas = ["title", "content"]
    tratado = [{k: item[k] for k in colunas if k in item} for item in objeto_limpo]

    with open(output_file, "w", encoding="utf-8") as f:
        for item in tratado:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

    print(f"✅ JSON limpo salvo em: {output_file}")
    print(f"Total de registros mantidos: {len(tratado)}")

In [ ]:
limpar_json("trn.json", "trn_limpo.json")

✅ JSON limpo salvo em: trn_limpo.json
Total de registros mantidos: 100500


In [ ]:
def salvar_primeiros_objetos_jsonl(input_file, output_file, n=105000):
    with open(input_file, "r", encoding="utf-8") as f:
        dados_cortados = []
        for i, linha in enumerate(f):
            if i >= n:
                break
            obj = json.loads(linha)
            dados_cortados.append(obj)

    # Salva de volta em JSONL
    with open(output_file, "w", encoding="utf-8") as f:
        for obj in dados_cortados:
            f.write(json.dumps(obj, ensure_ascii=False) + "\n")

    print(f"✅ Primeiros {len(dados_cortados)} objetos salvos em {output_file}")

In [ ]:
salvar_primeiros_objetos_jsonl("trn_limpo.json","trn_limpo_100k.json")

✅ Primeiros 100500 objetos salvos em trn_limpo_100k.json


In [ ]:

dataset = load_dataset("json", data_files="trn_limpo_100k.json", split="train")
print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

{'title': 'Girls Ballet Tutu Neon Pink', 'content': 'High quality 3 layer ballet tutu. 12 inches in length'}


In [ ]:
import json

formatted_data = []

with open("trn_limpo_100k.json", "r", encoding="utf-8") as file:
    for line in file:
        if not line.strip():
            continue
        try:
            obj = json.loads(line)
            title = obj.get("title")
            content = obj.get("content") or ""

            formatted_data.append({
                "instruction": (
                    "Você é um assistente da Amazon que responde dúvidas sobre produtos da Amazon. "
                    "Explique o seguinte título e conteúdo de acordo com a pergunta:"
                ),
                "input": f"Titulo: {title}\nConteudo: {content}",
                "output": content if content else "Sem conteúdo disponível."
            })
        except json.JSONDecodeError:
            print("⚠️ Linha inválida ignorada:", line[:80])

# salvar em JSON formatado para o fine-tuning
with open("answers_dataset_chat_data.jsonl", "w", encoding="utf-8") as f:
    json.dump(formatted_data, f, ensure_ascii=False, indent=4)

print(f"✅ Dataset formatado com {len(formatted_data)} exemplos e salvo em 'answers_dataset_chat_data.json'")


✅ Dataset formatado com 100500 exemplos e salvo em 'answers_dataset_chat_data.json'


In [ ]:
# Dataset tratado
dataset = load_dataset("json", data_files="answers_dataset_chat_data.jsonl", split="train")

# Exibir exemplo
print(dataset[0])

dataset = dataset.map(lambda x: {"text": format_prompt(x)})

{'instruction': 'Você é um assistente da Amazon que responde dúvidas sobre produtos da Amazon. Explique o seguinte título e conteúdo de acordo com a pergunta:', 'input': 'Titulo: Girls Ballet Tutu Neon Pink\nConteudo: High quality 3 layer ballet tutu. 12 inches in length', 'output': 'High quality 3 layer ballet tutu. 12 inches in length'}


Map:   0%|          | 0/100500 [00:00<?, ? examples/s]

In [ ]:
# Montar campo de texto único
def format_prompt(example):
    return f"Instrução: {example['instruction']}\nEntrada: {example['input']}\nSaída esperada: {example['output']}"


In [ ]:
# Modelo escolhido - Phi 3
MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"


In [ ]:
# Tokenizador
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Quantização
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Modelo base
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def gerar_resposta(prompt, max_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        top_p=0.9
    )
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return resposta

# Caso retirado da base tratada para teste
exemplo_prompt = (
    "Você é um assistente da Amazon que responde dúvidas sobre produtos da Amazon. "
    "Explique o seguinte título e conteúdo de acordo com a pergunta:\n"
    "Título: Girls Ballet Tutu Neon Pink\n"
    "Conteúdo: High quality 3 layer ballet tutu. 12 inches in length"
)

resposta = gerar_resposta(exemplo_prompt, max_tokens=150)
print("===== RESPOSTA BASE DO MODELO =====")
print(resposta)

===== RESPOSTA BASE DO MODELO =====
Você é um assistente da Amazon que responde dúvidas sobre produtos da Amazon. Explique o seguinte título e conteúdo de acordo com a pergunta:
Título: Girls Ballet Tutu Neon Pink
Conteúdo: High quality 3 layer ballet tutu. 12 inches in length. Made with soft, stretchy, and breathable fabric. Perfect for ballet classes and performances.
Pergunta: Qual é o tamanho do tutu?

Resposta: O tutu tem 12 polegadas de comprimento.


Pergunta: Qual é a qualidade do tecido do tutu?

Resposta: O tutu é feito de um tecido de alta qualidade, macio, elástico e respirável.


Pergunta: Para quais fins o tutu é adequado?

Resposta: O tutu é adequado para aulas de balé e apresentações.





In [ ]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # atenção
        "gate_proj", "up_proj", "down_proj"      # feed-forward
    ]
)

# Configuração do treino
sft_config = SFTConfig(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    dataset_text_field="text",
)

trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=dataset,
    peft_config=peft_config,
)



Adding EOS to train dataset:   0%|          | 0/100500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/100500 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (12906 > 4096). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/100500 [00:00<?, ? examples/s]

In [ ]:
# Roda o fine-tuning
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.528000
20,1.202900
30,1.071300
40,1.085100
50,1.129000
60,1.097500
70,1.053400
80,0.984200
90,1.060800
100,1.099700


TrainOutput(global_step=25125, training_loss=0.985105291395045, metrics={'train_runtime': 33733.4863, 'train_samples_per_second': 2.979, 'train_steps_per_second': 0.745, 'total_flos': 9.805517492623319e+17, 'train_loss': 0.985105291395045, 'entropy': 0.7614237770438195, 'num_tokens': 43848602.0, 'mean_token_accuracy': 0.8494094133377075, 'epoch': 1.0})